In [1]:
from dash import dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os

# ✅ Proper config for images in notebook
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🔧 Tool Carousel"

# 📦 Tool categories list
TOOL_DIR = os.path.abspath("../assets/object/tool")
TOOL_ITEMS = sorted([
    item for item in os.listdir(TOOL_DIR)
    if os.path.isdir(os.path.join(TOOL_DIR, item))
])

# 📸 Collect image paths per tool
tool_images = {}
for item in TOOL_ITEMS:
    item_dir = os.path.join(TOOL_DIR, item)
    rel_dir = os.path.relpath(item_dir, os.path.abspath("../assets"))
    tool_images[item] = sorted([
        f"/assets/{rel_dir}/{img}"
        for img in os.listdir(item_dir)
        if img.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🔧 Tool Carousel", style={"textAlign": "center", "marginBottom": "30px"}),

    html.Div([
        html.Label("Select tool item:", style={
            "fontWeight": "normal", "display": "block", "marginBottom": "8px"
        }),
        dcc.Dropdown(
            id='item-dropdown',
            options=[{
                "label": item.replace("_", " ").capitalize(),
                "value": item
            } for item in TOOL_ITEMS],
            value=TOOL_ITEMS[0],
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Carousel display
@app.callback(
    Output('carousel-container', 'children'),
    Input('item-dropdown', 'value')
)
def update_carousel(item):
    images = tool_images.get(item, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])

    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginRight": "20px"}),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginLeft": "20px"})
        ], style={"textAlign": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px", "fontStyle": "italic"})
    ])

# 🔁 Arrow navigation
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Launch server
app.run(port=8124)


/home/sbarbu/miniconda3/envs/datalad-env/lib/python3.10/site-packages/dash/dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(
